## Environment Setting

In [1]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install dominate==2.4.0
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 6.6MB 27.4MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
     |████████████████████████████████| 276kB 9.0MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=dc9139093aa9d0d6316067025b8033c88a30aa5f1ac09f1b92fd8acfc785bef8
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip

## Clone the file from Gihub

In [2]:
!git clone https://github.com/ericsujw/InstColorization.git

Cloning into 'InstColorization'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 211 (delta 0), reused 2 (delta 0), pack-reused 208
Receiving objects: 100% (211/211), 6.11 MiB | 39.62 MiB/s, done.
Resolving deltas: 100% (78/78), done.


## Start Colorization

In [3]:
cd InstColorization/

/content/InstColorization


## Download the Model

In [4]:
!sh scripts/download_model.sh

Downloading...
Finish download.
Archive:  checkpoints.zip
   creating: checkpoints/coco_finetuned_mask_256_ffs/
  inflating: checkpoints/coco_finetuned_mask_256_ffs/latest_net_GComp.pth  
  inflating: checkpoints/coco_finetuned_mask_256_ffs/latest_net_G.pth  
  inflating: checkpoints/coco_finetuned_mask_256_ffs/latest_net_GF.pth  
   creating: checkpoints/coco_finetuned_mask_256/
  inflating: checkpoints/coco_finetuned_mask_256/latest_net_GComp.pth  
  inflating: checkpoints/coco_finetuned_mask_256/latest_net_G.pth  
  inflating: checkpoints/coco_finetuned_mask_256/latest_net_GF.pth  
   creating: checkpoints/siggraph_retrained/
  inflating: checkpoints/siggraph_retrained/latest_net_G.pth  


## Detect Object bounding box


Setting the Detectron2.

In [5]:
from os.path import join, isfile, isdir
from os import listdir
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from argparse import ArgumentParser

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

import torch

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

model_final_2d9806.pkl: 431MB [00:05, 73.6MB/s]                           


## Download davis dataset

Link: https://davischallenge.org/

In [6]:
!wget https://graphics.ethz.ch/Downloads/Data/Davis/DAVIS-data.zip
!unzip DAVIS-data.zip 

Streaming output truncated to the last 5000 lines.
  inflating: DAVIS/JPEGImages/480p/elephant/00052.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00031.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00019.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00068.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00078.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00032.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00055.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00023.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00053.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00018.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00004.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00020.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00067.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00026.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00057.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00071.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00028.jpg

In [11]:
!ls DAVIS/ImageSets/480p/val.txt

DAVIS/ImageSets/480p/val.txt


In [25]:
import re

with open('DAVIS/ImageSets/480p/val.txt', 'r') as f:
  contents = f.read()

contents = [content.split(' ')[0] for content in contents.split('\n')[:-1]]

val_set = set()
for content in contents:
  m = re.search('/JPEGImages/480p/([a-z]+?\-*[a-z]*?)/[0-9]+?.jpg', content)
  val_set.add(m.group(1))
val_set = list(val_set)

print(val_set)

['drift-chicane', 'scooter-black', 'blackswan', 'parkour', 'breakdance', 'libby', 'camel', 'drift-straight', 'goat', 'dance-twirl', 'paragliding-launch', 'horsejump-high', 'car-shadow', 'car-roundabout', 'cows', 'bmx-trees', 'kite-surf', 'motocross-jump', 'dog', 'soapbox']


In [26]:
import os
import shutil

root_src = "DAVIS/JPEGImages/480p"
root_dst = "original_dataset"

if not os.path.exists(root_dst):
  os.mkdir(root_dst)

for folder in val_set:
  for filename in os.listdir(os.path.join(root_src, folder)):
    src = os.path.join(os.path.join(root_src, folder), filename)
    dst = os.path.join(root_dst, "{}_{}".format(folder, filename))
    shutil.copy(src, dst)

In [28]:
print(sorted(os.listdir(root_dst)))

['blackswan_00000.jpg', 'blackswan_00001.jpg', 'blackswan_00002.jpg', 'blackswan_00003.jpg', 'blackswan_00004.jpg', 'blackswan_00005.jpg', 'blackswan_00006.jpg', 'blackswan_00007.jpg', 'blackswan_00008.jpg', 'blackswan_00009.jpg', 'blackswan_00010.jpg', 'blackswan_00011.jpg', 'blackswan_00012.jpg', 'blackswan_00013.jpg', 'blackswan_00014.jpg', 'blackswan_00015.jpg', 'blackswan_00016.jpg', 'blackswan_00017.jpg', 'blackswan_00018.jpg', 'blackswan_00019.jpg', 'blackswan_00020.jpg', 'blackswan_00021.jpg', 'blackswan_00022.jpg', 'blackswan_00023.jpg', 'blackswan_00024.jpg', 'blackswan_00025.jpg', 'blackswan_00026.jpg', 'blackswan_00027.jpg', 'blackswan_00028.jpg', 'blackswan_00029.jpg', 'blackswan_00030.jpg', 'blackswan_00031.jpg', 'blackswan_00032.jpg', 'blackswan_00033.jpg', 'blackswan_00034.jpg', 'blackswan_00035.jpg', 'blackswan_00036.jpg', 'blackswan_00037.jpg', 'blackswan_00038.jpg', 'blackswan_00039.jpg', 'blackswan_00040.jpg', 'blackswan_00041.jpg', 'blackswan_00042.jpg', 'blackswan

Let's create a bounding box folder to save our prediction results.

In [29]:
input_dir = "original_dataset"
image_list = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
output_npz_dir = "{0}_bbox".format(input_dir)
if os.path.isdir(output_npz_dir) is False:
    print('Create path: {0}'.format(output_npz_dir))
    os.makedirs(output_npz_dir)

Create path: original_dataset_bbox


Here we simply take L channel as our input and make sure that we can get consistent box prediction results even though the original image is color images.

In [30]:
for image_path in image_list:
    img = cv2.imread(join(input_dir, image_path))
    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    l_stack = np.stack([l_channel, l_channel, l_channel], axis=2)
    outputs = predictor(l_stack)
    save_path = join(output_npz_dir, image_path.split('.')[0])
    pred_bbox = outputs["instances"].pred_boxes.to(torch.device('cpu')).tensor.numpy()
    pred_scores = outputs["instances"].scores.cpu().data.numpy()
    np.savez(save_path, bbox = pred_bbox, scores = pred_scores)

In [32]:
!ls original_dataset_bbox | wc -l

1376


## Colorize Images

We first set up some libraries and options

In [33]:
import sys
import time
from options.train_options import TestOptions
from models import create_model

import torch
from tqdm import tqdm_notebook

from fusion_dataset import Fusion_Testing_Dataset
from util import util
import multiprocessing
multiprocessing.set_start_method('spawn', True)

torch.backends.cudnn.benchmark = True

sys.argv = [sys.argv[0]]
opt = TestOptions().parse()

In [35]:
opt.test_img_dir = 'original_dataset'
opt.results_img_dir = 'dataset_results'

Then we need to create a results folder to save our predicted color images and read the dataset loader.

In [36]:
save_img_path = "dataset_results"
if os.path.isdir(save_img_path) is False:
    print('Create path: {0}'.format(save_img_path))
    os.makedirs(save_img_path)
opt.batch_size = 1
dataset = Fusion_Testing_Dataset(opt, -1)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=opt.batch_size)

dataset_size = len(dataset)
print('#Testing images = %d' % dataset_size)

#Testing images = 1376


Load the pre-trained model.

In [37]:
model = create_model(opt)
model.setup_to_test('coco_finetuned_mask_256_ffs')

initialize network with normal
initialize network with normal
initialize network with normal
model [FusionModel] was created
load Fusion model from checkpoints/coco_finetuned_mask_256_ffs/latest_net_GF.pth


Start to colorize every images in `dataset_loader`.

In [40]:
cd InstColorization

[Errno 2] No such file or directory: 'InstColorization'
/content/InstColorization


In [42]:
with torch.no_grad():
  count_empty = 0
  for data_raw in tqdm_notebook(dataset_loader):
      data_raw['full_img'][0] = data_raw['full_img'][0].cuda()
      if data_raw['empty_box'][0] == 0:
          data_raw['cropped_img'][0] = data_raw['cropped_img'][0].cuda()
          box_info = data_raw['box_info'][0]
          box_info_2x = data_raw['box_info_2x'][0]
          box_info_4x = data_raw['box_info_4x'][0]
          box_info_8x = data_raw['box_info_8x'][0]
          cropped_data = util.get_colorization_data(data_raw['cropped_img'], opt, ab_thresh=0, p=opt.sample_p)
          full_img_data = util.get_colorization_data(data_raw['full_img'], opt, ab_thresh=0, p=opt.sample_p)
          model.set_input(cropped_data)
          model.set_fusion_input(full_img_data, [box_info, box_info_2x, box_info_4x, box_info_8x])
          model.forward()
      else:
          count_empty += 1
          full_img_data = util.get_colorization_data(data_raw['full_img'], opt, ab_thresh=0, p=opt.sample_p)
          model.set_forward_without_box(full_img_data)
      model.save_current_imgs(join(save_img_path, data_raw['file_id'][0] + '.png'))
  print('{0} images without bounding boxes'.format(count_empty))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))



11 images without bounding boxes


## Colorize Images without bbox

In [43]:
save_img_path = "dataset_results_without_bbox"
if os.path.isdir(save_img_path) is False:
    print('Create path: {0}'.format(save_img_path))
    os.makedirs(save_img_path)
opt.batch_size = 1
dataset = Fusion_Testing_Dataset(opt, -1)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=opt.batch_size)

dataset_size = len(dataset)
print('#Testing images = %d' % dataset_size)

Create path: dataset_results_without_bbox
#Testing images = 1376


In [44]:
with torch.no_grad():
  for data_raw in tqdm_notebook(dataset_loader):
      data_raw['full_img'][0] = data_raw['full_img'][0].cuda()
      full_img_data = util.get_colorization_data(data_raw['full_img'], opt, ab_thresh=0, p=opt.sample_p)
      model.set_forward_without_box(full_img_data)
      model.save_current_imgs(join(save_img_path, data_raw['file_id'][0] + '.png'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [50]:
!ls dataset_results

blackswan_00000.png	  drift-chicane_00015.png
blackswan_00001.png	  drift-chicane_00016.png
blackswan_00002.png	  drift-chicane_00017.png
blackswan_00003.png	  drift-chicane_00018.png
blackswan_00004.png	  drift-chicane_00019.png
blackswan_00005.png	  drift-chicane_00020.png
blackswan_00006.png	  drift-chicane_00021.png
blackswan_00007.png	  drift-chicane_00022.png
blackswan_00008.png	  drift-chicane_00023.png
blackswan_00009.png	  drift-chicane_00024.png
blackswan_00010.png	  drift-chicane_00025.png
blackswan_00011.png	  drift-chicane_00026.png
blackswan_00012.png	  drift-chicane_00027.png
blackswan_00013.png	  drift-chicane_00028.png
blackswan_00014.png	  drift-chicane_00029.png
blackswan_00015.png	  drift-chicane_00030.png
blackswan_00016.png	  drift-chicane_00031.png
blackswan_00017.png	  drift-chicane_00032.png
blackswan_00018.png	  drift-chicane_00033.png
blackswan_00019.png	  drift-chicane_00034.png
blackswan_00020.png	  drift-chicane_00035.png
blackswan_00021.png	  drift-chican

## Save frames as well as videos

In [51]:
root = "DAVIS/JPEGImages/480p"
root_results = "dataset_colorization"
root_results_without_bb = "dataset_colorization_without_bb"
root_grayscale = "dataset_grayscale"

for dir in [root_results, root_results_without_bb, root_grayscale]:
  if not os.path.exists(dir):
    os.mkdir(dir)

for folder in val_set:
  for dir in [root_results, root_results_without_bb, root_grayscale]:
    if not os.path.exists(os.path.join(dir, folder)):
      os.mkdir(os.path.join(dir, folder))

  for filename in sorted(os.listdir(os.path.join(root, folder))):
    # Dataset results with bbox and grayscale images
    img = cv2.imread(os.path.join('original_dataset', "{}_{}".format(folder, filename)))
    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, _, _ = cv2.split(lab_image)

    img = cv2.imread(os.path.join('dataset_results', "{}_{}".format(folder, filename))[:-4] +'.png')
    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    _, a_pred, b_pred = cv2.split(lab_image)
    a_pred = cv2.resize(a_pred, (l_channel.shape[1], l_channel.shape[0]))
    b_pred = cv2.resize(b_pred, (l_channel.shape[1], l_channel.shape[0]))
    gray_color = np.ones_like(a_pred) * 128

    gray_image = cv2.cvtColor(np.stack([l_channel, gray_color, gray_color], 2), cv2.COLOR_LAB2BGR)
    color_image = cv2.cvtColor(np.stack([l_channel, a_pred, b_pred], 2), cv2.COLOR_LAB2BGR)

    cv2.imwrite(os.path.join(root_results, "{}/{}".format(folder, filename)), color_image)
    cv2.imwrite(os.path.join(root_grayscale, "{}/{}".format(folder, filename)), gray_image)

    # Dataset results without bbox
    img = cv2.imread(os.path.join('dataset_results_without_bbox', "{}_{}".format(folder, filename))[:-4] +'.png')
    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    _, a_pred, b_pred = cv2.split(lab_image)
    a_pred = cv2.resize(a_pred, (l_channel.shape[1], l_channel.shape[0]))
    b_pred = cv2.resize(b_pred, (l_channel.shape[1], l_channel.shape[0]))
    color_image = cv2.cvtColor(np.stack([l_channel, a_pred, b_pred], 2), cv2.COLOR_LAB2BGR)
    cv2.imwrite(os.path.join(root_results_without_bb, "{}/{}".format(folder, filename)), color_image)

In [53]:
!ls dataset_colorization_without_bb

blackswan   car-roundabout  dog		    horsejump-high  paragliding-launch
bmx-trees   car-shadow	    drift-chicane   kite-surf	    parkour
breakdance  cows	    drift-straight  libby	    scooter-black
camel	    dance-twirl     goat	    motocross-jump  soapbox


In [54]:
!ls dataset_colorization

blackswan   car-roundabout  dog		    horsejump-high  paragliding-launch
bmx-trees   car-shadow	    drift-chicane   kite-surf	    parkour
breakdance  cows	    drift-straight  libby	    scooter-black
camel	    dance-twirl     goat	    motocross-jump  soapbox


In [56]:
!ls dataset_grayscale

blackswan   car-roundabout  dog		    horsejump-high  paragliding-launch
bmx-trees   car-shadow	    drift-chicane   kite-surf	    parkour
breakdance  cows	    drift-straight  libby	    scooter-black
camel	    dance-twirl     goat	    motocross-jump  soapbox


In [57]:
import os
import moviepy.video.io.ImageSequenceClip

image_folder_root = 'dataset_colorization'
video_dir = 'video_colorization'
fps = 20

if not os.path.exists(video_dir):
  os.mkdir(video_dir)

for folder in os.listdir(image_folder_root):

  image_folder = os.path.join(image_folder_root, folder)

  image_files = sorted([image_folder+'/'+img for img in os.listdir(image_folder) if img.endswith(".jpg")])
  clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)

  clip.write_videofile(os.path.join(video_dir, folder + '.mp4'))

[MoviePy] >>>> Building video video_colorization/paragliding-launch.mp4
[MoviePy] Writing video video_colorization/paragliding-launch.mp4


100%|██████████| 80/80 [00:02<00:00, 39.03it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization/paragliding-launch.mp4 

[MoviePy] >>>> Building video video_colorization/soapbox.mp4
[MoviePy] Writing video video_colorization/soapbox.mp4


100%|██████████| 99/99 [00:02<00:00, 38.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization/soapbox.mp4 

[MoviePy] >>>> Building video video_colorization/kite-surf.mp4
[MoviePy] Writing video video_colorization/kite-surf.mp4


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization/breakdance.mp4 

[MoviePy] >>>> Building video video_colorization/dog.mp4
[MoviePy] Writing video video_colorization/dog.mp4


100%|██████████| 60/60 [00:01<00:00, 52.31it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization/dog.mp4 

[MoviePy] >>>> Building video video_colorization/libby.mp4
[MoviePy] Writing video video_colorization/libby.mp4


100%|██████████| 49/49 [00:00<00:00, 72.03it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization/libby.mp4 



In [58]:
import os
import moviepy.video.io.ImageSequenceClip

image_folder_root = 'dataset_colorization_without_bb'
video_dir = 'video_colorization_without_bb'
fps = 20

if not os.path.exists(video_dir):
  os.mkdir(video_dir)

for folder in os.listdir(image_folder_root):

  image_folder = os.path.join(image_folder_root, folder)

  image_files = sorted([image_folder+'/'+img for img in os.listdir(image_folder) if img.endswith(".jpg")])
  clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)

  clip.write_videofile(os.path.join(video_dir, folder + '.mp4'))

[MoviePy] >>>> Building video video_colorization_without_bb/paragliding-launch.mp4
[MoviePy] Writing video video_colorization_without_bb/paragliding-launch.mp4


100%|██████████| 80/80 [00:01<00:00, 41.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_without_bb/paragliding-launch.mp4 

[MoviePy] >>>> Building video video_colorization_without_bb/soapbox.mp4
[MoviePy] Writing video video_colorization_without_bb/soapbox.mp4


100%|██████████| 99/99 [00:02<00:00, 39.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_without_bb/soapbox.mp4 

[MoviePy] >>>> Building video video_colorization_without_bb/kite-surf.mp4
[MoviePy] Writing video video_colorization_without_bb/kite-surf.mp4


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_without_bb/breakdance.mp4 

[MoviePy] >>>> Building video video_colorization_without_bb/dog.mp4
[MoviePy] Writing video video_colorization_without_bb/dog.mp4


100%|██████████| 60/60 [00:01<00:00, 51.49it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_without_bb/dog.mp4 

[MoviePy] >>>> Building video video_colorization_without_bb/libby.mp4
[MoviePy] Writing video video_colorization_without_bb/libby.mp4


100%|██████████| 49/49 [00:00<00:00, 73.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_without_bb/libby.mp4 



In [59]:
import os
import moviepy.video.io.ImageSequenceClip

image_folder_root = 'dataset_grayscale'
video_dir = 'video_grayscale'
fps = 20

if not os.path.exists(video_dir):
  os.mkdir(video_dir)

for folder in os.listdir(image_folder_root):

  image_folder = os.path.join(image_folder_root, folder)

  image_files = sorted([image_folder+'/'+img for img in os.listdir(image_folder) if img.endswith(".jpg")])
  clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)

  clip.write_videofile(os.path.join(video_dir, folder + '.mp4'))

[MoviePy] >>>> Building video video_grayscale/paragliding-launch.mp4
[MoviePy] Writing video video_grayscale/paragliding-launch.mp4


100%|██████████| 80/80 [00:01<00:00, 46.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_grayscale/paragliding-launch.mp4 

[MoviePy] >>>> Building video video_grayscale/soapbox.mp4
[MoviePy] Writing video video_grayscale/soapbox.mp4


100%|██████████| 99/99 [00:02<00:00, 43.27it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_grayscale/soapbox.mp4 

[MoviePy] >>>> Building video video_grayscale/kite-surf.mp4
[MoviePy] Writing video video_grayscale/kite-surf.mp4


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: video_grayscale/breakdance.mp4 

[MoviePy] >>>> Building video video_grayscale/dog.mp4
[MoviePy] Writing video video_grayscale/dog.mp4


100%|██████████| 60/60 [00:01<00:00, 54.60it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_grayscale/dog.mp4 

[MoviePy] >>>> Building video video_grayscale/libby.mp4
[MoviePy] Writing video video_grayscale/libby.mp4


100%|██████████| 49/49 [00:00<00:00, 70.12it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_grayscale/libby.mp4 



In [60]:
import os
import moviepy.video.io.ImageSequenceClip

image_folder_root = 'DAVIS/JPEGImages/480p'
video_dir = 'video_original'
fps = 20

if not os.path.exists(video_dir):
  os.mkdir(video_dir)

for folder in val_set:
  image_folder = os.path.join(image_folder_root, folder)

  image_files = sorted([image_folder+'/'+img for img in os.listdir(image_folder) if img.endswith(".jpg")])
  clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)

  clip.write_videofile(os.path.join(video_dir, folder + '.mp4'))

[MoviePy] >>>> Building video video_original/drift-chicane.mp4
[MoviePy] Writing video video_original/drift-chicane.mp4


100%|██████████| 52/52 [00:00<00:00, 75.90it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_original/drift-chicane.mp4 

[MoviePy] >>>> Building video video_original/scooter-black.mp4
[MoviePy] Writing video video_original/scooter-black.mp4


100%|██████████| 44/44 [00:00<00:00, 94.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_original/scooter-black.mp4 

[MoviePy] >>>> Building video video_original/blackswan.mp4
[MoviePy] Writing video video_original/blackswan.mp4


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: video_original/motocross-jump.mp4 

[MoviePy] >>>> Building video video_original/dog.mp4
[MoviePy] Writing video video_original/dog.mp4


100%|██████████| 60/60 [00:01<00:00, 52.95it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_original/dog.mp4 

[MoviePy] >>>> Building video video_original/soapbox.mp4
[MoviePy] Writing video video_original/soapbox.mp4


100%|██████████| 99/99 [00:02<00:00, 41.73it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_original/soapbox.mp4 



## Save

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
!zip -r dataset_colorization.zip dataset_colorization
!zip -r dataset_colorization_without_bb.zip dataset_colorization_without_bb
!zip -r dataset_grayscale.zip dataset_grayscale
!cp dataset_colorization.zip dataset_colorization_without_bb.zip dataset_grayscale.zip '../drive/MyDrive/colorization'

  adding: dataset_colorization/ (stored 0%)
  adding: dataset_colorization/paragliding-launch/ (stored 0%)
  adding: dataset_colorization/paragliding-launch/00043.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00009.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00066.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00003.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00006.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00010.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00041.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00062.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00073.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00005.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00079.jpg (deflated 1%)
  adding: dataset_colorization/paragliding-launch/00012.jpg (deflated 1%)
  add

In [65]:
!cp -r video_colorization video_colorization_without_bb video_grayscale video_original '../drive/MyDrive/colorization'

In [83]:
import cv2
import numpy as np

out='concat'

if not os.path.exists(out):
  os.mkdir(out)
for folder in val_set:
  if not os.path.exists(os.path.join(out, folder)):
    os.mkdir(os.path.join(out, folder))
  for filename in sorted(os.listdir(os.path.join(root, folder))):
    imgs = []
    for dir in [root, root_grayscale, root_results, root_results_without_bb]:
      imgs.append(cv2.imread(os.path.join(os.path.join(dir, folder), filename)))

      vis = np.concatenate(imgs, axis=1)
    
    cv2.imwrite(os.path.join(os.path.join(out, folder), filename), vis)

In [86]:
import os
import moviepy.video.io.ImageSequenceClip

image_folder_root = out
video_dir = 'video_concat'
fps = 10

if not os.path.exists(video_dir):
  os.mkdir(video_dir)

for folder in val_set:
  image_folder = os.path.join(image_folder_root, folder)

  image_files = sorted([image_folder+'/'+img for img in os.listdir(image_folder) if img.endswith(".jpg")])
  clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)

  clip.write_videofile(os.path.join(video_dir, folder + '.mp4'))

[MoviePy] >>>> Building video video_concat/drift-chicane.mp4
[MoviePy] Writing video video_concat/drift-chicane.mp4


100%|██████████| 52/52 [00:02<00:00, 18.54it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_concat/drift-chicane.mp4 

[MoviePy] >>>> Building video video_concat/scooter-black.mp4
[MoviePy] Writing video video_concat/scooter-black.mp4


100%|██████████| 44/44 [00:01<00:00, 25.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_concat/scooter-black.mp4 

[MoviePy] >>>> Building video video_concat/blackswan.mp4
[MoviePy] Writing video video_concat/blackswan.mp4


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: video_concat/motocross-jump.mp4 

[MoviePy] >>>> Building video video_concat/dog.mp4
[MoviePy] Writing video video_concat/dog.mp4


100%|██████████| 60/60 [00:04<00:00, 12.15it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_concat/dog.mp4 

[MoviePy] >>>> Building video video_concat/soapbox.mp4
[MoviePy] Writing video video_concat/soapbox.mp4


100%|██████████| 99/99 [00:10<00:00,  9.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_concat/soapbox.mp4 



In [87]:
!cp -r video_concat '../drive/MyDrive/colorization'